### PROJECT - SQL-BASED ANALYSIS OF PRODUCT SALES
## Project Summary  

This project involved analyzing music store sales data using SQL and Python (SQLite + Pandas).  
The goal was to explore revenue trends, customer behavior, and product performance.  

Key steps included:  
- **Data Cleaning & Exploration** – Checked tables, relationships, and key attributes.  
- **Sales Analysis** – Calculated total sales by joining invoices, invoice lines, and tracks.  
- **Top Products & Albums** – Identified best-selling tracks and albums.  
- **Regional Insights** – Compared revenue across different countries.  
- **Time-based Trends** – Analyzed monthly revenue to detect seasonal patterns.

In [ ]:
#import required libraries
import numpy as np 
import pandas as pd 
import sqlite3
import matplotlib.pyplot as plt

In [ ]:
#connect to SQLite
conn = sqlite3.connect(r"C:\Users\ud\Desktop\Chinook_Sqlite.sqlite")

#show all tables in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

In [ ]:
#list of all tables
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';", conn
)

for table in tables['name']: #loop through tables
    print(f"\n Table: {table}")
    cols = pd.read_sql_query(f"PRAGMA table_info({table});", conn)
    print(cols[['name', 'type']])  # only show column names and data type

In [ ]:
#top selling products
query= """ 
SELECT t.Name AS Track,
       a.Title AS Album,
       SUM(il.UnitPrice * il.Quantity) AS Total_Sales
FROM InvoiceLine AS il
JOIN Track AS t 
ON il.TrackId = t.TrackId
JOIN Album AS a 
ON t.AlbumId = a.AlbumId
GROUP BY t.TrackId
ORDER BY Total_Sales DESC
LIMIT 10;
"""
top_products = pd.read_sql_query(query, conn)
print(top_products)

### Insights from Top-Selling Products
- TV series soundtracks dominate sales, with shows like The Office and Heroes contributing multiple top-selling tracks. This indicates a strong customer interest in popular culture and TV-related contents. 
- Most bestsellers are priced at $3.98, showing that this price point is highly effective in driving sales  
- Customers favor purchasing individual tracks/episodes rather than entire albums, suggesting convenience and selectivity drive buying behavior. 
- Repeat sales from shows across multiple seasons (e.g., *The Office*) highlight strong fan bases. 

In [ ]:
#revenue by Region
query = """
SELECT c.Country,
       ROUND(SUM(i.Total), 2) AS Revenue
FROM Invoice AS i
JOIN Customer AS c 
ON i.CustomerId = c.CustomerId
GROUP BY c.Country
ORDER BY Revenue DESC
LIMIT 10;
"""
revenue_region = pd.read_sql_query(query, conn)
print(revenue_region)

### Revenue by Region Insights: 
- The USA leads as the strongest market with the highest revenue ($523.06), followed by Canada, making North America the dominant region overall.
- European countries like France, Germany, and the UK are key secondary markets, providing steady income.
- Emerging markets such as Brazil and India show notable sales, highlighting opportunities for growth.
- Chile records the lowest revenue among the top 10, suggesting weaker demand in smaller regions.

In [ ]:
#monthly performance
query = """
SELECT strftime('%Y - %m', InvoiceDate) AS Month,
       ROUND(SUM(Total), 2) AS Revenue
FROM Invoice
GROUP BY Month
ORDER BY Month;
"""
monthly_perf = pd.read_sql_query(query, conn)
print(monthly_perf)

### Monthly Performance Insights 
- Revenue stayed relatively stable across most months.
- Significant spikes occurred in Jan 2010, Apr & Jun 2011, Aug 2012, and Nov 2013. These peaks suggest seasonal or promotional boosts.
- Sharp declines were seen in Nov 2011 and Feb 2013, indicating potential dips in customer activity or lower sales campaigns.

In [ ]:
#Use of RANK() function
query= """
SELECT CustomerId,
       ROUND(SUM(Total), 2) AS Revenue,
       RANK() OVER (ORDER BY SUM(Total) DESC) AS Rank
FROM Invoice
GROUP BY CustomerId
LIMIT 10;
"""
top_customers = pd.read_sql_query(query, conn)
print(top_customers)

In [ ]:
# Plot for monthly revenue
plt.figure(figsize=(10,5))
plt.plot(monthly_perf["Month"], monthly_perf["Revenue"], marker='o')
plt.xticks(rotation=90)
plt.title("Monthly Revenue Performance")
plt.xlabel("Month")
plt.ylabel("Revenue")
plt.show()


In [ ]:
import seaborn as sns

#Bar Chart for Top selling products
plt.figure(figsize=(10,6))
sns.barplot(
    x="Total_Sales", 
    y="Track", 
    data=top_products, 
    hue="Track",      #link palette to track
    palette="viridis", 
    legend=False        
)
plt.title("Top 10 Selling Tracks", fontsize=14)
plt.xlabel("Total Sales")
plt.ylabel("Track")
plt.show()

In [ ]:
# Bar Chart for Revenue by Country 
plt.figure(figsize=(10,6))
sns.barplot(
    x="Revenue", 
    y="Country", 
    data=revenue_region, 
    hue="Country",       #link palette to Country
    palette="magma", 
    legend=False         
)
plt.title("Revenue by Country", fontsize=14)
plt.xlabel("Revenue")
plt.ylabel("Country")
plt.show()
